# Extracting directions, Classification and Removal
We search for directions corresponding to a concept in hidden layer activations.
We use several different methods:
* One Prompt
* Logistic Regression
* Principal Component Analysis (PCA)
* Class Means
* K-Means
* Random Direction as a baseline

We check how well the directions correlate with the concept we care about by using them to separate the test data.

We furthermore check how much information about the concept is left after removing information along the directions.

In [ ]:
import sys
import importlib
import os
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import gc

from collections import defaultdict
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
my_path = "/data/ann_kathrin_dombrowski/ICE/ice_baseline/"

In [ ]:
# import my modules

# Append the absolute path to sys.path
sys.path.append(os.path.join(my_path, 'modules'))
import wrapping
import utils
importlib.reload(wrapping)
importlib.reload(utils)
from wrapping import WrappedModel
from utils import *


In [ ]:
results_dir = f'{my_path}results/'
plots_dir = f'{my_path}plots/'
data_dir = f'/data/ann_kathrin_dombrowski/ethics/utilitarianism/' # download from https://github.com/hendrycks/ethics

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)


## Load Data

We show results on the utility dataset. You can download it from [here](https://people.eecs.berkeley.edu/~hendrycks/ethics.tar). Just copy the downloaded folder into the data folder.

In [ ]:
def load_util_sentences(split="test"):
    data = pd.read_csv(data_dir + f'util_{split}.csv', header=None)
    sentences = []
    for d in data.iloc:
        sentences.append([d[0], d[1]])
    return np.array(sentences)



X_train = load_util_sentences(split='train')
X_test = load_util_sentences(split='test')
print(X_train.shape, X_test.shape)

# per default always the first sentence is the sentence with higher utility
# this function swaps the sentences with probability 0.5
def get_data(rng, data):
    m = len(data)
    labels = rng.integers(0, 2, size=m)
    data = data[np.arange(m), [1-labels, labels]].T
    return data, labels

rng = np.random.default_rng(0)
X_train, y_train = get_data(rng, X_train)
X_test, y_test = get_data(rng, X_test)


### Example sentences
The label is one if the first sentence is more utilitarian than the second sentence and zero otherwise.

In [ ]:
for i in range(5):
    print(f"sentence 1: {X_train[i, 0]}")
    print(f"sentence 2: {X_train[i, 1]}")
    print(f"  -> label: {y_train[i]}")


## Load Model

Use a huggingface [access token](https://huggingface.co/docs/hub/security-tokens) and load the Llama-2-7b-chat model.

In [ ]:
# enter your authentication token from huggingface and press enter to access the models
auth_token = input()

In [ ]:
# load model
model_name = "Llama-2-7b-chat-hf"
model_path = f"meta-llama/{model_name}"
precision = torch.bfloat16
model = AutoModelForCausalLM.from_pretrained(model_path, token=auth_token, device_map="auto").to(device=DEVICE, dtype=precision)
model.eval()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, token=auth_token, device_map="auto")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' 

In [ ]:
num_layers = model.config.num_hidden_layers

## Get hidden activations
We use the custom WrappedModel class to get the internal activations. There are other ways to do this (for example with transformer lens or baukit).
We first add a prompt to each sentence to push the model into considering the concept that we aim to extract.
We then run each sentence through the model and save the hidden activations in each layer.
We get the outputs of the residual stream (the decoder block) per default.

In [ ]:
num_layers = model.config.num_hidden_layers
layer_ids = np.arange(0, num_layers) # which layers do we want to extract the hidden states from?

hidden_dim = model.config.hidden_size
block_name = "decoder_block" # should be one of ['self_attn', 'mlp', 'input_layernorm', 'post_attention_layernorm', 'decoder_block']
token_pos = -1 # at which token do we want to extract the hidden states? -1 means the last token.
batch_size = 128
# WRAP MODEL
# wrap the model in a class that allows to access the hidden states
wrapped_model = WrappedModel(model, tokenizer)
# make sure nothing is wrapped from previous runs
wrapped_model.unwrap()
# wrap the block you want to wrap
wrapped_model.wrap_block(layer_ids, block_name=block_name)

In [ ]:
format_prompt = 'Consider the amount of happiness in the following scenario:\n"{scenario}"\nThe amount of happiness in the scenario is '

In [ ]:
def batchify(lst, batch_size):
    """Yield successive batch_size chunks from lst."""
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

def calc_hidden_states(wrapped_model, sentences, format_prompt, batch_size=128, token_pos=-1):
    hidden_states = {}
    for sentence_batch in tqdm(batchify(sentences, batch_size), total=len(sentences)//batch_size):
        wrapped_model.reset()
        gc.collect()

        formatted_sentences = [format_prompt.format_map({'scenario': s}) for s in sentence_batch]        
        # get activations
        _ = wrapped_model.run_prompt(formatted_sentences) # this saves the hidden states in the wrapped_model object
        # output is a dictionary with layer_ids as keys and the hidden states of the token_pos as values
        outputs = wrapped_model.get_activations(layer_ids, block_name=block_name, token_pos=token_pos)
        for key, values in outputs.items():
            values = values.detach().float().cpu().numpy()
            # Check if the key already exists in hidden_states
            if key in hidden_states:
                # Concatenate the tensors along axis 0 and update hidden_states
                hidden_states[key] = np.concatenate((hidden_states[key], values), axis=0)
            else:
                # If the key doesn't exist in hidden_states, simply assign the values
                hidden_states[key] = values
    return hidden_states

In [ ]:
H0_train = calc_hidden_states(wrapped_model, X_train[:, 0], format_prompt, batch_size=batch_size, token_pos=token_pos)
H1_train = calc_hidden_states(wrapped_model, X_train[:, 1], format_prompt, batch_size=batch_size, token_pos=token_pos)
H0_test = calc_hidden_states(wrapped_model, X_test[:, 0], format_prompt, batch_size=batch_size, token_pos=token_pos)
H1_test = calc_hidden_states(wrapped_model, X_test[:, 1], format_prompt, batch_size=batch_size, token_pos=token_pos)

We split the training set again, since we want an untouched part of the training set for our removal code.
For some methods we use the differences between contrastive pairs. We normalize all data.

In [ ]:
n_dir_find = int(H0_train[0].shape[0]/2)
start_clf = n_dir_find
# n_dir_find = 2500

H0_train_s, H1_train_s, H_train_s, H_test = {}, {}, {}, {}
H0_train_clf, H1_train_clf, H_train_clf = {}, {}, {}
y_train_s = y_train[:n_dir_find]
y_train_clf = y_train[start_clf:]
for layer in H0_train.keys():
    H0_train_s[layer], H1_train_s[layer] = H0_train[layer][:n_dir_find], H1_train[layer][:n_dir_find]
    H_train_s[layer] = H0_train[layer][:n_dir_find]-H1_train[layer][:n_dir_find]
    H0_train_clf[layer], H1_train_clf[layer] = H0_train[layer][start_clf:], H1_train[layer][start_clf:]
    H_train_clf[layer] = H0_train[layer][start_clf:]-H1_train[layer][start_clf:]
    H_test[layer] = H0_test[layer]-H1_test[layer]

In [ ]:
# normalizing data
scalers = {}
dscalers = {}
for layer in layer_ids:

    scaler = StandardScaler()
    scaler.fit(np.concatenate([H0_train_s[layer], H1_train_s[layer]], axis=0))
    scalers[layer] = scaler
    H0_train_s[layer] = scaler.transform(H0_train_s[layer])
    H1_train_s[layer] = scaler.transform(H1_train_s[layer])
    H0_train_clf[layer] = scaler.transform(H0_train_clf[layer])
    H1_train_clf[layer] = scaler.transform(H1_train_clf[layer])
    H0_test[layer] = scaler.transform(H0_test[layer])
    H1_test[layer] = scaler.transform(H1_test[layer])

    
    dscalers[layer] = StandardScaler()
    dscalers[layer].fit(H_train_s[layer])
    H_train_s[layer] = dscalers[layer].transform(H_train_s[layer])
    H_train_clf[layer] = dscalers[layer].transform(H_train_clf[layer])
    H_test[layer] = dscalers[layer].transform(H_test[layer])

# Finding directions using different methods

We find the directions using the hidden representation of our formatted sentences directly or after taking differences between contrastive pairs. To ensure that all directions point towards positive utility we project the training data on the un oriented direction and find the correct coefficient for the orientation using logistic regression.

In [ ]:
directions = {}

In [ ]:
# one prompt method

H0_prompt = calc_hidden_states(wrapped_model, ["Love"], format_prompt, batch_size=batch_size, token_pos=token_pos)
H1_prompt = calc_hidden_states(wrapped_model, ["Hate"], format_prompt, batch_size=batch_size, token_pos=token_pos)

directions["OnePrompt"] = {}

for layer in tqdm(layer_ids):
    H0_prompt[layer] = scalers[layer].transform(H0_prompt[layer])
    H1_prompt[layer] = scalers[layer].transform(H1_prompt[layer])
    direction = H0_prompt[layer]-H1_prompt[layer]
    direction = dscalers[layer].transform(direction)
    directions["OnePrompt"][layer] = direction.squeeze()

In [ ]:

directions["OnePrompt"][0].shape

In [ ]:
# random method
directions["Random"] = {}
num_random_directions = 10
for layer in tqdm(layer_ids):
    direction = np.random.normal(0.0, 1.0, size=(num_random_directions, hidden_dim))
    for i in range(direction.shape[0]):
        # project data onto direction
        Htr_i = np.dot(H_train_s[layer], direction[i].squeeze().T)
        lr = LogisticRegression(solver='liblinear').fit(Htr_i.reshape(-1, 1), y_train_s)
        coeff = np.sign(lr.coef_).squeeze()
        direction[i] = coeff*direction[i]
    directions["Random"][layer] = direction

In [ ]:
# Logistic regression
directions["LogReg"] = {}
for layer in tqdm(layer_ids):
    lr = LogisticRegression(solver='liblinear', C=1e-2, random_state=0, max_iter=50).fit(H_train_s[layer], y_train_s)
    coeff = np.sign(lr.coef_).squeeze()
    directions["LogReg"][layer] = coeff

In [ ]:
# Class means
directions["ClassMeans"] = {}
for layer in tqdm(layer_ids):
    direction = H_train_s[layer][y_train_s==1].mean(axis=0) - H_train_s[layer][y_train_s==0].mean(axis=0)
    directions["ClassMeans"][layer] = direction

In [ ]:
# PCA on differences of contrastive pairs
directions["PCA_diffs"] = {}
for layer in tqdm(layer_ids):
    pca = PCA(n_components=1, whiten=True).fit(H_train_s[layer])
    direction = pca.components_.squeeze()
    temp = pca.transform(H_train_s[layer])
    lr = LogisticRegression(solver='liblinear').fit(temp, y_train_s)
    coeff = np.sign(lr.coef_).squeeze()
    directions["PCA_diffs"][layer] = coeff*direction

In [ ]:
# PCA without taking differences
directions["PCA"] = {}
for layer in tqdm(layer_ids):
    pca = PCA(n_components=1, whiten=True).fit(np.concatenate([H0_train_s[layer], H1_train_s[layer]], axis=0))
    direction = pca.components_.squeeze()
    temp = pca.transform(H_train_s[layer])
    lr = LogisticRegression(solver='liblinear').fit(temp, y_train_s)
    coeff = np.sign(lr.coef_).squeeze()
    directions["PCA"][layer] = coeff*direction

In [ ]:
# K-means
directions["K-Means"] = {}
for layer in tqdm(layer_ids):
    kmeans = KMeans(n_clusters=2, n_init=10).fit(np.concatenate([H0_train_s[layer], H1_train_s[layer]], axis=0))
    direction = kmeans.cluster_centers_[0] - kmeans.cluster_centers_[1]
    # project onto direction
    temp = np.dot(H_train_s[layer], direction.squeeze().T)
    lr = LogisticRegression(solver='liblinear').fit(temp.reshape(-1, 1), y_train_s)
    coeff = np.sign(lr.coef_).squeeze()
    directions["K-Means"][layer] = direction*coeff

In [ ]:
fname = results_dir + 'utility_directions_{model_name}.pkl'
with open(fname, 'wb') as f:
    pickle.dump(directions, f, pickle.HIGHEST_PROTOCOL)

### Cosine similarity between directions

In [ ]:
dfs = []
# calculate cosine similarity between directions
for layer in tqdm(range(num_layers)):
    temp = {key : directions[key][layer] for key in directions.keys()}
    temp.pop("Random")
    for key in temp.keys():
        temp[key] = temp[key].squeeze()
    df = pd.DataFrame.from_dict(temp, orient='index')
    cosine_sim_matrix = cosine_similarity(df.values)
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=df.index, columns=df.index)
    dfs.append(cosine_sim_df)

In [ ]:
sns.heatmap(dfs[20], annot=True)
plt.savefig(plots_dir + f'utility_cossine_{model_name}_selection.pdf', dpi=300, bbox_inches='tight')

# Classification - Test for Correlation
How well can the found directions separate the data? We test on differences of the hidden representation of the test data.

In [ ]:
def classification(H, y, direction):
    Hte_i = np.dot(H, direction.T)
    accte = ((Hte_i > 0) == y).sum()/len(y)
    return accte

In [ ]:
test_accs = {}
for method in tqdm(directions.keys()):
    test_accs[method] = {}
    for layer in layer_ids:
        if method == "Random":
            temp = 0
            random_runs = directions[method][layer].shape[0]
            for i in range(random_runs):
                temp += classification(H_test[layer], y_test, directions[method][layer][i])
            test_accs[method][layer] = temp/random_runs
        else:
            test_accs[method][layer] = classification(H_test[layer], y_test, directions[method][layer])


In [ ]:
plot_lines(test_accs, "test accuracy", save_path=plots_dir + f'baselines_utility_{model_name}.pdf')